In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
n = 100000
p = pd.read_pickle(f"../../../app/db/p_{n}.pkl")
q = pd.read_pickle(f"../../../app/db/q_{n}.pkl")
user_ratings = pd.read_csv("../../../data/cleaned/user_ratings.csv").head(n)
print(p.shape)
print(q.shape)
print(user_ratings.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../../../app/db/p_100000.pkl'

In [ ]:
def rmse(actual: np.ndarray, predicted: np.ndarray) -> float:
    return mean_squared_error(actual, predicted, squared=False)

In [ ]:
user_ratings["Latent rating"] = np.einsum("ij, ij->i", p.loc[user_ratings["Username"]].values, q.loc[user_ratings["BGGId"]].values)

In [ ]:
user_ratings["Global mean"] = user_ratings["Rating"].mean()
user_ratings["User mean"] = user_ratings.groupby("Username")["Rating"].transform("mean")
user_ratings["Game mean"] = user_ratings.groupby("BGGId")["Rating"].transform("mean")
user_ratings["User bias"] = user_ratings["User mean"] - user_ratings["Global mean"]
user_ratings["Game bias"] = user_ratings["Game mean"] - user_ratings["Global mean"]
user_ratings["Naive rating"] = user_ratings["Global mean"] + user_ratings["User bias"] + user_ratings["Game bias"]
user_ratings.head(100)

In [ ]:
print(rmse(user_ratings["Rating"], user_ratings["Latent rating"]))
print(rmse(user_ratings["Rating"], user_ratings["Naive rating"]))